##### LibraryDatabase

In [1]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect('LibraryDatabase.db')
cursor = conn.cursor()

###### Create Tables

In [2]:
# Create USERS table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        full_name TEXT NOT NULL,
        enabled TEXT CHECK(enabled IN ('t', 'f')),
        LastLogin DATETIME
    );
    ''')

# Create ADDRESS Table
cursor.execute('''
  CREATE TABLE IF NOT EXISTS Addresses (
        address_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        street TEXT NOT NULL,
        city TEXT NOT NULL,
        state TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES Users(id)
    );
    ''')

# Create Books Table
cursor.execute('''
 CREATE TABLE IF NOT EXISTS Books (
        book_id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author TEXT NOT NULL,
        publishedDate DATETIME,
        ISBN TEXT UNIQUE NOT NULL
    );
    ''')

# Create CHECKOUTS Table
cursor.execute('''
 CREATE TABLE IF NOT EXISTS Checkouts(
        checkout_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        book_id INTEGER NOT NULL,
        checkout_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        return_date DATETIME,
        FOREIGN KEY (user_id) REFERENCES Users(id),
        FOREIGN KEY (book_id) REFERENCES Books(id)
    );
    ''')

# Create REVIEWS Table
cursor.execute('''
  CREATE TABLE IF NOT EXISTS Reviews (
        review_id INTEGER PRIMARY KEY AUTOINCREMENT,
        book_id INTEGER NOT NULL,
        reviewer_name TEXT,
        Content TEXT,
        rating INTEGER CHECK (rating BETWEEN 1 AND 5),
        published_date DATETIME,
        FOREIGN KEY (book_id) REFERENCES Books(id)
    );
    ''')

# Create LOGS Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Logs (
        log_id INTEGER PRIMARY KEY AUTOINCREMENT,
        checkout_id INTEGER NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (checkout_id) REFERENCES Checkouts(checkout_id)
    );
    ''')

# Create Trigger to log checkouts
cursor.execute('''
    CREATE TRIGGER IF NOT EXISTS log_checkout
    AFTER INSERT ON Checkouts
    FOR EACH ROW
    BEGIN
        INSERT INTO Logs (checkout_id) VALUES (NEW.checkout_id);
    END;''')

conn.commit()

###### Insert Values

In [3]:
# Insert data into Users
cursor.execute('''
    INSERT INTO Users (id, full_name, enabled, lastLogin) VALUES 
    (1, 'John Smith', 'f', '2017-10-25 10:26:10.015152'),
    (2, 'Alice Walker', 't', '2017-10-25 10:26:50.295461'),
    (3, 'Harry Potter', 't', '2017-10-25 10:26:50.295461'),
    (5, 'Jane Smith', 't', '2017-10-25 10:36:43.324015');
''')

# Insert data into Addresses
cursor.execute('''
   INSERT INTO Addresses (user_id, street, city, state) VALUES 
(1, '1 Market Street', 'San Francisco', 'CA'),
(2, '2 Elm Street', 'San Francisco', 'CA'),
(3, '3 Main Street', 'Boston', 'MA');
''')

# Insert values into the Books table
cursor.execute('''
    INSERT INTO Books (title, author, publishedDate, ISBN) VALUES 
    ('My First SQL book', 'Mary Parker', '2012-02-22 12:08:17.320053-03', '981483029127'),
    ('My Second SQL book', 'John Mayer', '1972-07-03 09:22:45.050088-07', '857300923713'),
    ('My Third SQL book', 'Cary Flint', '2015-10-18 14:05:44.547516-07', '523120967812');
''')

# Insert data into Checkouts
cursor.execute('''
    INSERT INTO Checkouts (user_id, book_id, checkout_date, return_date) VALUES 
    (1, 1, '2017-10-15 14:43:18.095143-07', NULL),
    (1, 2, '2017-10-05 16:22:44.593188-07', '2017-10-13 13:05:12.673382-05'),
    (2, 2, '2017-10-15 11:11:24.994973-07', '2017-10-22 17:47:10.407569-07'),
    (5, 3, '2017-10-15 09:27:07.215217-07', NULL);
''')

# Insert data into Reviews
cursor.execute('''
    INSERT INTO Reviews (book_id, reviewer_name, Content, rating, published_date) VALUES 
    (1, 'John Smith', 'Excellent book', 4, '2017-12-10 05:50:11.127281-02'),
    (2, 'John Smith', 'Best SQL book', 5, '2017-10-13 15:05:12.673382-05'),
    (2, 'Alice Walker', '10/10 recommended', 1, '2017-10-22 23:47:10.407569-07');
''')

conn.commit()


###### Display Tables

In [4]:
# Getting a list of all tables in the database.
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the names and contents of each table.
for table_name in tables:
    table_name = table_name[0] 
    print(f"\n{table_name} Table:")
    
    # Getting column names
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    column_names = [column[1] for column in columns] 
    
    # Printing the column names
    print(", ".join(column_names))
    
    # Fetch and print each row from the table
    cursor.execute(f"SELECT * FROM {table_name};")
    rows = cursor.fetchall()
    
    # Printing each row
    for row in rows:
        print(row)
    
conn.close()



Users Table:
id, full_name, enabled, LastLogin
(1, 'John Smith', 'f', '2017-10-25 10:26:10.015152')
(2, 'Alice Walker', 't', '2017-10-25 10:26:50.295461')
(3, 'Harry Potter', 't', '2017-10-25 10:26:50.295461')
(5, 'Jane Smith', 't', '2017-10-25 10:36:43.324015')

sqlite_sequence Table:
name, seq
('Users', 5)
('Addresses', 3)
('Books', 3)
('Logs', 4)
('Checkouts', 4)
('Reviews', 3)

Addresses Table:
address_id, user_id, street, city, state
(1, 1, '1 Market Street', 'San Francisco', 'CA')
(2, 2, '2 Elm Street', 'San Francisco', 'CA')
(3, 3, '3 Main Street', 'Boston', 'MA')

Books Table:
book_id, title, author, publishedDate, ISBN
(1, 'My First SQL book', 'Mary Parker', '2012-02-22 12:08:17.320053-03', '981483029127')
(2, 'My Second SQL book', 'John Mayer', '1972-07-03 09:22:45.050088-07', '857300923713')
(3, 'My Third SQL book', 'Cary Flint', '2015-10-18 14:05:44.547516-07', '523120967812')

Checkouts Table:
checkout_id, user_id, book_id, checkout_date, return_date
(1, 1, 1, '2017-10-15

##### Q1: Find the title, authors and the isbn of the books that ‘John Smith’ has checked out.

In [5]:
# Connecting to the SQLite database
conn = sqlite3.connect('LibraryDatabase.db')
cursor = conn.cursor()
# Query to find books checked out by 'John Smith'
cursor.execute('''
    SELECT Books.title, Books.author, Books.ISBN
    FROM Users
    JOIN Checkouts ON Users.id = Checkouts.user_id
    JOIN Books ON Checkouts.book_id = Books.book_id
    WHERE Users.full_name = 'John Smith';
''')

print("----Books checked out by John Smith: ----")
# Fetch and display the results
results = cursor.fetchall()
for row in results:
    print(f"Title: {row[0]}, Author: {row[1]}, ISBN: {row[2]}")

# closing the connection
conn.close()


----Books checked out by John Smith: ----
Title: My First SQL book, Author: Mary Parker, ISBN: 981483029127
Title: My Second SQL book, Author: John Mayer, ISBN: 857300923713


##### Q2: Find all reviewers for the book “My Third SQL book”.

In [6]:
# Connecting to the SQLite database
conn = sqlite3.connect('LibraryDatabase.db')
cursor = conn.cursor()

# Query to find all reviewers for the book "My Third SQL book"
cursor.execute('''
    SELECT Reviews.reviewer_name
    FROM Books
    JOIN Reviews ON Books.book_id = Reviews.book_id
    WHERE Books.title = 'My Third SQL book';
''')

# Fetch the results
results = cursor.fetchall()

print("---- REVIEWS ----")
# Check if there are results
if results:
    for row in results:
        print(f"Reviewer: {row[0]}")
else:
    print("0 reviews")

# closing the connection
conn.close()



---- REVIEWS ----
0 reviews


##### Q3: Find the users that have no books checked out

In [7]:
# Connecting to the SQLite database
conn = sqlite3.connect('LibraryDatabase.db')
cursor = conn.cursor()

# Query to find users with no books checked out
cursor.execute('''
    SELECT Users.full_name
    FROM Users
    LEFT JOIN Checkouts ON Users.id = Checkouts.user_id
    WHERE Checkouts.user_id IS NULL;
''')

# Fetch and display the results
results = cursor.fetchall()

print("--- books with no checkouts ---")
# Check if there are results
if results:
    for row in results:
        print(f"{row[0]}")
else:
    print("All users have books checked out.")


--- books with no checkouts ---
Harry Potter


####  Q4: Write a SQL query to show all the records of the logs table

In [8]:
# Query to fetch all records from the Logs table
cursor.execute('SELECT * FROM Logs;')

# Fetch and display the results
results = cursor.fetchall()

# Check if there are results
if results:
    print("Logs Table:")
    for row in results:
        print(row)
else:
    print("No records found in the Logs table.")


Logs Table:
(1, 1, '2024-11-04 19:35:41')
(2, 2, '2024-11-04 19:35:41')
(3, 3, '2024-11-04 19:35:41')
(4, 4, '2024-11-04 19:35:41')


## Sending the sql query to pandas function

In [9]:
#import pandas
import pandas as pd
# Connecting to the SQLite database
conn = sqlite3.connect('LibraryDatabase.db')
cursor = conn.cursor()
# Query each table using pandas and display results
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_names = tables['name'].tolist()
table_data = {table_name: pd.read_sql_query(f"SELECT * FROM {table_name};", conn) for table_name in table_names}
# Display all tables
for table_name, data in table_data.items():
    print(f"\n--- {table_name} Table ---")
    print(data)




--- Users Table ---
   id     full_name enabled                   LastLogin
0   1    John Smith       f  2017-10-25 10:26:10.015152
1   2  Alice Walker       t  2017-10-25 10:26:50.295461
2   3  Harry Potter       t  2017-10-25 10:26:50.295461
3   5    Jane Smith       t  2017-10-25 10:36:43.324015

--- sqlite_sequence Table ---
        name  seq
0      Users    5
1  Addresses    3
2      Books    3
3       Logs    4
4  Checkouts    4
5    Reviews    3

--- Addresses Table ---
   address_id  user_id           street           city state
0           1        1  1 Market Street  San Francisco    CA
1           2        2     2 Elm Street  San Francisco    CA
2           3        3    3 Main Street         Boston    MA

--- Books Table ---
   book_id               title       author                  publishedDate  \
0        1   My First SQL book  Mary Parker  2012-02-22 12:08:17.320053-03   
1        2  My Second SQL book   John Mayer  1972-07-03 09:22:45.050088-07   
2        3   My Th

###### Q1: Find the title, authors and the isbn of the books that ‘John Smith’ has checked out.

In [10]:
# 1. Books checked out by 'John Smith'
books_checked_out = pd.read_sql_query('''
    SELECT Books.title, Books.author, Books.ISBN
    FROM Users
    JOIN Checkouts ON Users.id = Checkouts.user_id
    JOIN Books ON Checkouts.book_id = Books.book_id
    WHERE Users.full_name = 'John Smith';
''', conn)
print("\n--- Books checked out by John Smith ---")
print(books_checked_out)



--- Books checked out by John Smith ---
                title       author          ISBN
0   My First SQL book  Mary Parker  981483029127
1  My Second SQL book   John Mayer  857300923713


###### Q2: Find all reviewers for the book “My Third SQL book”.

In [11]:
# 2. Reviewers for "My Third SQL book"
reviewers = pd.read_sql_query('''
    SELECT Reviews.reviewer_name
    FROM Books
    JOIN Reviews ON Books.book_id = Reviews.book_id
    WHERE Books.title = 'My Third SQL book';
''', conn)
print("\n--- Reviewers for 'My Third SQL book' ---")
print(reviewers)



--- Reviewers for 'My Third SQL book' ---
Empty DataFrame
Columns: [reviewer_name]
Index: []


###### Q3: Find the users that have no books checked out

In [12]:
# 3. Users with no books checked out
users_no_books = pd.read_sql_query('''
    SELECT Users.full_name
    FROM Users
    LEFT JOIN Checkouts ON Users.id = Checkouts.user_id
    WHERE Checkouts.user_id IS NULL;
''', conn)
print("\n--- Users with no books checked out ---")
print(users_no_books)


--- Users with no books checked out ---
      full_name
0  Harry Potter


##### Q4: Write a SQL query to show all the records of the logs table

In [13]:
# 4. All records from the Logs table
logs_table = pd.read_sql_query("SELECT * FROM Logs;", conn)
print("\n--- Logs Table ---")
print(logs_table)

# Close connection
conn.close()


--- Logs Table ---
   log_id  checkout_id            timestamp
0       1            1  2024-11-04 19:35:41
1       2            2  2024-11-04 19:35:41
2       3            3  2024-11-04 19:35:41
3       4            4  2024-11-04 19:35:41
